In [1]:
import pandas as pd

# Load the Excel file
file_path = '/Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/data/DRONE_METADATA_CEAB_falseWKT.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
df.head()

,dwc:eventID,dwc:eventType,dwc:countryCode,dwc:waterBody,dwc:locality,dwc:decimalLongitude,dwc:decimalLatitude,dwc:footprintWKT,dwc:geodeticDatum,dwc:institutionCode,...,IMAGES,ALTITUDE,AREA,REP_ERROR,TIME,GSD,GCP,GROUND TRUTH,PILOT,FUNDING
0,20180000BLANES,drone survey,ES,Mediterranean,BLANES,2.801750,41.674150,NaN,EPSG:4326,CEAB-CSIC,...,70,NaN,NaN,NaN,NaN,0,NO,NO,SANTAMARIA J,NaN
1,20180000CAPCREUS,drone survey,ES,Mediterranean,CAIALS,3.295839,42.285847,NaN,EPSG:4326,CEAB-CSIC,...,36,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,NaN
2,20190724BLANES,drone survey,ES,Mediterranean,BLANES,2.816607,41.681846,NaN,EPSG:4326,CEAB-CSIC,...,57,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT
3,20191011BLANES,drone survey,ES,Mediterranean,BLANES,2.816482,41.681898,NaN,EPSG:4326,CEAB-CSIC,...,79,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT
4,20200930BLANES,drone survey,ES,Mediterranean,BLANES,2.816059,41.682516,NaN,EPSG:4326,CEAB-CSIC,...,65,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT


In [2]:
import geopandas as gpd
from shapely.geometry import Point

# Create a geometry column from decimalLatitude and decimalLongitude
geometry = [Point(xy) for xy in zip(df['dwc:decimalLongitude'], df['dwc:decimalLatitude'])]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Display the first few rows of the GeoDataFrame
gdf.head()

,dwc:eventID,dwc:eventType,dwc:countryCode,dwc:waterBody,dwc:locality,dwc:decimalLongitude,dwc:decimalLatitude,dwc:footprintWKT,dwc:geodeticDatum,dwc:institutionCode,...,ALTITUDE,AREA,REP_ERROR,TIME,GSD,GCP,GROUND TRUTH,PILOT,FUNDING,geometry
0,20180000BLANES,drone survey,ES,Mediterranean,BLANES,2.801750,41.674150,NaN,EPSG:4326,CEAB-CSIC,...,NaN,NaN,NaN,NaN,0,NO,NO,SANTAMARIA J,NaN,POINT (2.80175 41.67415)
1,20180000CAPCREUS,drone survey,ES,Mediterranean,CAIALS,3.295839,42.285847,NaN,EPSG:4326,CEAB-CSIC,...,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,NaN,POINT (3.29584 42.28585)
2,20190724BLANES,drone survey,ES,Mediterranean,BLANES,2.816607,41.681846,NaN,EPSG:4326,CEAB-CSIC,...,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT,POINT (2.81661 41.68185)
3,20191011BLANES,drone survey,ES,Mediterranean,BLANES,2.816482,41.681898,NaN,EPSG:4326,CEAB-CSIC,...,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT,POINT (2.81648 41.68190)
4,20200930BLANES,drone survey,ES,Mediterranean,BLANES,2.816059,41.682516,NaN,EPSG:4326,CEAB-CSIC,...,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT,POINT (2.81606 41.68252)


In [7]:
from shapely.geometry import Polygon

# Function to create a square polygon with a given centroid and side length (in meters)
def create_square_polygon(centroid, side_length):
    half_side = side_length / 2
    x, y = centroid.x, centroid.y
    square_coords = [(x - half_side, y - half_side), (x + half_side, y - half_side),
                     (x + half_side, y + half_side), (x - half_side, y + half_side),
                     (x - half_side, y - half_side)]
    return Polygon(square_coords)

# Set the CRS for the original GeoDataFrame (assuming WGS84)
gdf = gdf.set_crs(epsg=4326)

# Convert the GeoDataFrame to a projected coordinate system (e.g., UTM) for accurate distance calculations
gdf_proj = gdf.to_crs(epsg=32633)  # Example EPSG code for UTM zone 33N, change as needed

# Create a new column 'square_wkt' with the WKT representation of the square polygons
gdf_proj['square_wkt'] = gdf_proj['geometry'].apply(lambda geom: create_square_polygon(geom, 200).wkt)

# Convert back to the original coordinate system
gdf['square_wkt'] = gdf_proj['square_wkt']

# Display the first few rows of the GeoDataFrame with the new column
gdf[['geometry', 'square_wkt']].head()

# Convert the square polygons back to the original coordinate system (EPSG:4326)
gdf['square_wkt'] = gdf_proj['square_wkt'].apply(lambda wkt: gpd.GeoSeries.from_wkt([wkt], crs=gdf_proj.crs).to_crs(epsg=4326).iloc[0].wkt)

# Display the first few rows of the GeoDataFrame with the converted polygons
gdf[['geometry', 'square_wkt']].head()

,geometry,square_wkt
0,POINT (2.80175 41.67415),"POLYGON ((2.800744618560536 41.67314311127887,..."
1,POINT (3.29584 42.28585),"POLYGON ((3.29481692617127 42.28484220844574, ..."
2,POINT (2.81661 41.68185),POLYGON ((2.8156012557587027 41.68083919592764...
3,POINT (2.81648 41.68190),POLYGON ((2.8154762573791414 41.68089119494588...
4,POINT (2.81606 41.68252),POLYGON ((2.8150532574393576 41.68150919060498...


In [8]:
# Add the square_wkt column to df under dwc:footprintWKT
df['dwc:footprintWKT'] = gdf['square_wkt']

# Display the first few rows of the updated dataframe
df.head()

,dwc:eventID,dwc:eventType,dwc:countryCode,dwc:waterBody,dwc:locality,dwc:decimalLongitude,dwc:decimalLatitude,dwc:footprintWKT,dwc:geodeticDatum,dwc:institutionCode,...,IMAGES,ALTITUDE,AREA,REP_ERROR,TIME,GSD,GCP,GROUND TRUTH,PILOT,FUNDING
0,20180000BLANES,drone survey,ES,Mediterranean,BLANES,2.801750,41.674150,"POLYGON ((2.800744618560536 41.67314311127887,...",EPSG:4326,CEAB-CSIC,...,70,NaN,NaN,NaN,NaN,0,NO,NO,SANTAMARIA J,NaN
1,20180000CAPCREUS,drone survey,ES,Mediterranean,CAIALS,3.295839,42.285847,"POLYGON ((3.29481692617127 42.28484220844574, ...",EPSG:4326,CEAB-CSIC,...,36,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,NaN
2,20190724BLANES,drone survey,ES,Mediterranean,BLANES,2.816607,41.681846,POLYGON ((2.8156012557587027 41.68083919592764...,EPSG:4326,CEAB-CSIC,...,57,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT
3,20191011BLANES,drone survey,ES,Mediterranean,BLANES,2.816482,41.681898,POLYGON ((2.8154762573791414 41.68089119494588...,EPSG:4326,CEAB-CSIC,...,79,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT
4,20200930BLANES,drone survey,ES,Mediterranean,BLANES,2.816059,41.682516,POLYGON ((2.8150532574393576 41.68150919060498...,EPSG:4326,CEAB-CSIC,...,65,NaN,NaN,NaN,NaN,0,NO,NO,BOADA J,GENCAT


In [9]:
# Save the modified GeoDataFrame to the Excel file
df.to_excel(file_path, index=False)